In [1]:
import os

In [2]:
list_file = list(os.walk('./input'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
for file in list_file:
    if '改价表' in file:
        price = './input/' + file
    elif 'position' in file:
        position = './input/' + file
    elif 'colorless' in file:
        this = './input/' + file

print('price:         ' + price)
print('position:      ' + position)
print('this:          ' + this)

price:         ./input/改价表 4-23.xlsx
position:      ./input/1.position.xlsx
this:          ./input/am-colorless.xlsx


In [3]:
import pandas as pd

In [4]:
df_price = pd.read_excel(price,
                         sheet_name=0,
                         header=0,
                         dtype=str).fillna('')[['SKU', '数量']]

df_price = df_price[df_price['SKU'] != ''].reset_index(drop=True)

df_price['数量'] = df_price['数量'].astype(int)
df_price = df_price.groupby(df_price['SKU'], as_index=False).sum()

df_price

,SKU,数量
0,USABS0L700-C,1
1,USABS11690-C,1
2,USABS12673-C,2
3,USABS13150-C,2
4,USABS1440-C,1
...,...,...
1983,USWS47500-C,1
1984,USWSP44302S0-4-YC,1
1985,USWSP44303R0-2-YC,1
1986,USWSP54722B1-4-YC,1


In [5]:
df_position = pd.read_excel(position,
                            header=0,
                            dtype=str).fillna('')

df_position

,主料,SKU,row
0,USABS00021-C,USABS00021-C,1
1,USABS00117B-C,USABS00117B-C,2
2,USABS00217B-C,USABS00217B-C,3
3,USABS00317B-C,USABS00317B-C,4
4,USABS00417-C,USABS00417-C,5
...,...,...,...
106446,USGS6310TNZB-C,USGS6310TNZ,28287
106447,USIC02680L-6-C,USIC02680-6,31054
106448,USIC02680L-C,USIC02680,31055
106449,USIC2A366Z-C,USIC2A366,31162


In [6]:
print(df_price.columns)
print(df_position.columns)

df_price = pd.merge(left=df_price,
                    right=df_position,
                    how='left',
                    left_on='SKU',
                    right_on='SKU').fillna('')

df_price

Index(['SKU', '数量'], dtype='object')
Index(['主料', 'SKU', 'row'], dtype='object')


,SKU,数量,主料,row
0,USABS0L700-C,1,USABS0L700-C,38
1,USABS11690-C,1,USABS11690-C,111
2,USABS12673-C,2,USABS12673-C,131
3,USABS13150-C,2,USABS13150-C,146
4,USABS1440-C,1,USABS1440-C,180
...,...,...,...,...
1983,USWS47500-C,1,,
1984,USWSP44302S0-4-YC,1,,
1985,USWSP44303R0-2-YC,1,,
1986,USWSP54722B1-4-YC,1,,


In [7]:
# 找出有position关系的数据
df_temp = df_price[df_price['row'] != '']
df_temp = df_temp[['row', '数量']]
df_temp = df_temp.groupby([df_temp['row']], as_index=False).sum()

df_temp

,row,数量
0,1003,1
1,10068,1
2,10069,1
3,10146,1
4,10147,1
...,...,...
1511,9943,1
1512,9957,1
1513,9958,1
1514,997,1


In [8]:
print(df_price.columns)
print(df_temp.columns)

# 更新有position关系的数据
df_price = pd.merge(left=df_price,
                    right=df_temp,
                    how='left',
                    left_on='row',
                    right_on='row',
                    suffixes=('', '_temp')).fillna('')

df_price['数量'] = [amount if amount_temp == '' else amount_temp for amount, amount_temp in zip(df_price['数量'].tolist(), df_price['数量_temp'].tolist())]

df_price = df_price[['SKU', '数量']]

df_price

Index(['SKU', '数量', '主料', 'row'], dtype='object')
Index(['row', '数量'], dtype='object')


,SKU,数量
0,USABS0L700-C,1.0
1,USABS11690-C,1.0
2,USABS12673-C,2.0
3,USABS13150-C,2.0
4,USABS1440-C,1.0
...,...,...
1983,USWS47500-C,1.0
1984,USWSP44302S0-4-YC,1.0
1985,USWSP44303R0-2-YC,1.0
1986,USWSP54722B1-4-YC,1.0


In [9]:
df_this = pd.read_excel(this,
                        header=1,
                        usecols='C',
                        dtype=str).fillna('')

df_this

,SKU
0,USBCB72928BLR-C
1,USPSP14045D-C
2,USBCB75352ZLR-C
3,USBCB08180ZLR-C
4,USPSP01008D-C
...,...
29608,USSTW924638-C
29609,USSTW924647-C
29610,USSTW925504-C
29611,USSTW925507-C


In [10]:
print(df_this.columns)
print(df_price.columns)

df_this = pd.merge(left=df_this,
                   right=df_price,
                   how='left',
                   left_on='SKU',
                   right_on='SKU').fillna(0)

df_this

Index(['SKU'], dtype='object')
Index(['SKU', '数量'], dtype='object')


,SKU,数量
0,USBCB72928BLR-C,0.0
1,USPSP14045D-C,0.0
2,USBCB75352ZLR-C,0.0
3,USBCB08180ZLR-C,0.0
4,USPSP01008D-C,0.0
...,...,...
29608,USSTW924638-C,0.0
29609,USSTW924647-C,0.0
29610,USSTW925504-C,0.0
29611,USSTW925507-C,0.0


In [12]:
from datetime import datetime

In [13]:
df_this.to_excel('./补充-' + datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

print('Done ~')

Done ~
